In [1]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

In [2]:
class SimpleCNN:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1
		# first CONV => RELU => BN layer set
		model.add(Conv2D(32, (3, 3), strides=(2, 2), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		# second CONV => RELU => BN layer set
		model.add(Conv2D(64, (3, 3), strides=(2, 2), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(128))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))
		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))
		# return the constructed network architecture
		return model

In [3]:
# import the necessary packages
from tensorflow.keras.losses import MSE
import tensorflow as tf

def generate_image_adversary_targeted(model, image, target_label, eps=2 / 255.0):
    # Cast the image to float32
    image = tf.cast(image, tf.float32)

    # Record gradients
    with tf.GradientTape() as tape:
        tape.watch(image)
        # Make prediction on the input image and compute the loss with the target label
        pred = model(image)
        loss = -MSE(target_label, pred)  # Negative MSE to maximize the distance

    # Calculate the gradients of the loss with respect to the image, compute the sign of the gradient
    gradient = tape.gradient(loss, image)
    signedGrad = tf.sign(gradient)

    # Construct the image adversary by subtracting the signed gradient
    adversary = (image - (signedGrad * eps)).numpy()
    return adversary


In [4]:
# import the necessary packages
#from pyimagesearch.simplecnn import SimpleCNN
# from pyimagesearch.fgsm import generate_image_adversary
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import numpy as np
import cv2

# load MNIST dataset and scale the pixel values to the range [0, 1]
print("[INFO] loading MNIST dataset...")
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX / 255.0
testX = testX / 255.0
# add a channel dimension to the images
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)
# one-hot encode our labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

# initialize our optimizer and model
print("[INFO] compiling model...")
opt = Adam(learning_rate=1e-3)
model = SimpleCNN.build(width=28, height=28, depth=1, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the simple CNN on MNIST
print("[INFO] training network...")
model.fit(trainX, trainY,
	validation_data=(testX, testY),
	batch_size=64,
	epochs=10,
	verbose=1)
# make predictions on the testing set for the model trained on
# non-adversarial images
(loss, acc) = model.evaluate(x=testX, y=testY, verbose=0)
print("[INFO] loss: {:.4f}, acc: {:.4f}".format(loss, acc))

for i in np.random.choice(np.arange(0, len(testX)), size=(10,)):
    image = testX[i]
    label = testY[i]

    # Define the target label (here we just pick class '1' for demonstration)
    target_label = np.zeros_like(label)
    target_label[1] = 1

    # Generate a targeted adversarial image
    adversary = generate_image_adversary_targeted(model, image.reshape(1, 28, 28, 1), target_label, eps=0.1)
    pred = model.predict(adversary)

    # scale both the original image and adversary to the range
    # [0, 255] and convert them to an unsigned 8-bit integers
    adversary = adversary.reshape((28, 28)) * 255
    adversary = np.clip(adversary, 0, 255).astype("uint8")
    image = image.reshape((28, 28)) * 255
    image = image.astype("uint8")
    # convert the image and adversarial image from grayscale to three
    # channel (so we can draw on them)
    image = np.dstack([image] * 3)
    adversary = np.dstack([adversary] * 3)
    # resize the images so we can better visualize them
    image = cv2.resize(image, (96, 96))
    adversary = cv2.resize(adversary, (96, 96))

    # determine the predicted label for both the original image and
    # adversarial image
    imagePred = label.argmax()
    adversaryPred = pred[0].argmax()
    color = (0, 255, 0)
    # if the image prediction does not match the adversarial
    # prediction then update the color
    if imagePred != adversaryPred:
        color = (0, 0, 255)
    # draw the predictions on the respective output images
    cv2.putText(image, str(imagePred), (2, 25),
        cv2.FONT_HERSHEY_SIMPLEX, 0.95, (0, 255, 0), 2)
    cv2.putText(adversary, str(adversaryPred), (2, 25),
        cv2.FONT_HERSHEY_SIMPLEX, 0.95, color, 2)
    # stack the two images horizontally and then show the original
    # image and adversarial image
    output = np.hstack([image, adversary])
    cv2.imshow("FGSM Adversarial Images", output)
    cv2.waitKey(0)

[INFO] loading MNIST dataset...
[INFO] compiling model...
[INFO] training network...


/Users/kimheejune/miniconda3/envs/aml/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.8836 - loss: 0.3897 - val_accuracy: 0.9780 - val_loss: 0.0695
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - accuracy: 0.9754 - loss: 0.0795 - val_accuracy: 0.9722 - val_loss: 0.0807
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.9825 - loss: 0.0588 - val_accuracy: 0.9861 - val_loss: 0.0444
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.9853 - loss: 0.0457 - val_accuracy: 0.9859 - val_loss: 0.0432
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.9874 - loss: 0.0385 - val_accuracy: 0.9870 - val_loss: 0.0399
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - accuracy: 0.9915 - loss: 0.0292 - val_accuracy: 0.9883 - val_loss: 0.0370
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - accuracy: 0.9915 - loss: 0.0277 - val_accuracy: 0.9882 - val_loss: 0.0360
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - accuracy: 0.9924 - loss: 0.0237 - 

: 